# Описание задачи


Ссылка на данные: https://ods.ai/competitions/nlp-receipts/data

Данные чеков ОФД содержат детальную информацию о тратах клиентов. Они помогают улучшать качество моделей кредитного скоринга и склонности к банковским продуктам, а также улучшать пользовательский опыт за счет структуризации трат клиентов в мобильном приложении. Однако работа с этим источником затрудняется его неструктурированностью: вся информация о купленном товаре лежит в одной строке произвольного формата.

В предположении что каждая чековая позиция описывает какой-либо товар, наименование этого товара, а также его бренд, являются главной информацией, которую можно извлечь из чека. По итогу задача структуризации этих данных ограничивается выделением и нормализацией брендов и товаров.

Участникам соревнования предоставляются два датасета с чековыми позициями, размеченный и неразмеченный:

В размеченном датасете для каждой чековой позиции указаны нормализованные бренды и товары входящие в нее в исходном виде.
В неразмеченном датасете даны только сами чековые позиции.

# Import

In [1]:
import pandas as pd
import numpy as np
import re

from gensim.models.fasttext import FastText
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

RAND = 10

In [2]:
train_supervised_path = 'data/train_supervised_dataset.csv'
train_unsupervised_path = 'data/train_unsupervised_dataset.csv'
test_path = 'data/test_dataset.csv'

In [3]:
df_sup = pd.read_csv(train_supervised_path).fillna("")

In [4]:
df_unsup = pd.read_csv(train_unsupervised_path).fillna("")

In [5]:
df_test = pd.read_csv(test_path).fillna("")

# Feature engineering

In [6]:
def clean_text(text: pd.Series) -> pd.Series:
    """
    Чистит текст
    """
    text = text.lower()
    #text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # удаляет пунктцацию
    text = re.sub(r'\w*\d+\w*', '', text) # удаляет цифры
    text = re.sub(r'[^\w\s]', ' ', text) # удаляет знаки
    text = re.sub(r'\b\S{1}\b', '', text) # удаляет слова из 1-й буквы
    text = re.sub(r'\b\S{2}\b', '', text) # удаляет слова из 2-х букв
    text = re.sub('\s{2,}', " ", text) # удаляет ненужные пробелы
    
    return text

df_sup['name'] = df_sup['name'].apply(clean_text)
df_unsup['name'] = df_unsup['name'].apply(clean_text)
df_test['name'] = df_test['name'].apply(clean_text)

In [10]:
# токенизирует
df_sup['tokens'] = df_sup['name'].str.lower().str.split()
df_unsup['tokens'] = df_unsup['name'].str.lower().str.split()
df_test["tokens"] = df_test["name"].str.lower().str.split()

In [ ]:
df = pd.concat([df_unsup, df_test, df_sup.drop(['good', 'brand'], axis=1)])

In [11]:
def apply_bio_tagging(row: pd.DataFrame) -> pd.Series:
    """
    По токенам чека и разметке (то есть выделенным товарам и брендам) строит BIO-теги
    """
    tokens = row["tokens"]
    good = row["good"].split(',')[0].split()
    brand = row["brand"].split(',')[0].split()
    tags = ['O'] * len(tokens)
    for i, token in enumerate(tokens):
        if len(good) > 0 and tokens[i:i + len(good)] == good:
            tags[i] = "B-GOOD"
            for j in range(i + 1, i + len(good)):
                tags[j] = "I-GOOD"
        if len(brand) > 0 and tokens[i:i + len(brand)] == brand:
            tags[i] = "B-BRAND"
            for j in range(i + 1, i + len(brand)):
                tags[j] = "I-BRAND"
                
    return tags

In [13]:
df_sup["tags"] = df_sup.apply(apply_bio_tagging, axis=1)

In [16]:
fst_model = FastText(df['tokens'],
                     vector_size=300,
                     window=3,
                     min_count=1,
                     sg=1,
                     alpha=0.1,
                     negative=10,
                     epochs=15)


KeyboardInterrupt



In [ ]:
def to_series(column: pd.Series) -> pd.Series:
    """
    Преобразует столбец в необходимый вид
    """
    token_list = column.to_list()
    flat_list = [item for sublist in token_list for item in sublist]
    tokens_series = pd.Series(flat_list)
    
    return tokens_series

In [ ]:
def make_datasets(data: pd.DataFrame) -> pd.DataFrame:
    """
    Создает новые датафреймы, на основе векторов и BIO-тегов
    """
    tags = to_series(df_sup["tags"])
    tokenss = to_series(df_sup["tokens"])
    dataset = tokenss.apply(lambda x: fst_model.wv[x])
    dtv_train = pd.concat([pd.DataFrame(dataset.to_list()), 
           pd.DataFrame(tags)], axis=1, ignore_index=True)
    dtv_train.rename(columns={300:"labels"}, inplace=True)

    good_train = dtv_train.loc[dtv_train['labels'].isin(["B-GOOD", "I-GOOD", "O"])]
    brand_train = dtv_train.loc[dtv_train['labels'].isin(["B-BRAND", "I-BRAND", "O"])]
    
    return good_train, brand_train

In [ ]:
good_train, brand_train = make_datasets(df_sup)

# Modeling (Catboost)

In [ ]:
X_good = good_train.drop("labels", axis=1)
y_good = good_train.labels

X_brand = brand_train.drop("labels", axis=1)
y_brand = brand_train.labels

X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_good,
                                                            y_good,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            random_state=RAND)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_brand,
                                                            y_brand,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            random_state=RAND)

eval_set_g = [(X_test_g, y_test_g)]
eval_set_b = [(X_test_b, y_test_b)]

In [ ]:
clf_g = CatBoostClassifier(random_state=RAND,
                           eval_metric='TotalF1',
                           loss_function='MultiClass')

clf_b = CatBoostClassifier(random_state=RAND,
                           eval_metric='TotalF1',
                           loss_function='MultiClass')

clf_g.fit(X_train_g,
          y_train_g,
          eval_set=eval_set_g,
          early_stopping_rounds=100, 
          use_best_model=True,
          verbose=False)

clf_b.fit(X_train_b,
          y_train_b,
          eval_set=eval_set_b,
          early_stopping_rounds=100, 
          use_best_model=True,
          verbose=False)

# Tuning

In [ ]:
grid = {
    'n_estimators': [1000],
    'learning_rate': np.linspace(0.01, 0.1, 5),
    #'boosting_type' : ['Ordered', 'Plain'],
    #'max_depth': list(range(3, 12)),
    #'l2_leaf_reg': np.logspace(-5, 2, 5),
    #'random_strength': list(range(10, 50, 5)),
    #'bootstrap_type': ["Bayesian", "Bernoulli", "MVS", "No"],
    #'border_count': [128, 254],
    #'grow_policy': ["SymmetricTree", "Depthwise", "Lossguide"],
    'random_state': [RAND]
}

In [ ]:
model_g = CatBoostClassifier(random_state=RAND,
                             eval_metric='TotalF1',
                             loss_function='MultiClass',
                             silent=True)
grid_search_result_g = model_g.randomized_search(grid,
                                              X=X_train_g,
                                              y=y_train_g, 
                                              verbose=False)

In [ ]:
model_b = CatBoostClassifier(random_state=RAND,
                             eval_metric='TotalF1',
                             loss_function='MultiClass',
                             silent=True)
grid_search_result_b = model_b.randomized_search(grid,
                                              X=X_train_b,
                                              y=y_train_b, 
                                              verbose=False)

In [ ]:
cat_best_g = grid_search_result_g['params']
cat_best_b = grid_search_result_b['params']

In [ ]:
cat_grid_g = CatBoostClassifier(**cat_best_g,
                                eval_metric='TotalF1')

cat_grid_b = CatBoostClassifier(**cat_best_b,
                                eval_metric='TotalF1')

cat_grid_g.fit(X_train_g,
               y_train_g,
               eval_set=eval_set_g,
               early_stopping_rounds=100, 
               use_best_model=True,
               verbose=False)

cat_grid_b.fit(X_train_b,
               y_train_b,
               eval_set=eval_set_b,
               early_stopping_rounds=100, 
               use_best_model=True,
               verbose=False)

# Create submit

In [ ]:
def create_submit_good(row: pd.Series) -> pd.Series:
    """
    Формирует предсказания товаров
    :param row: токены
    :return: предсказанные товары
    """
    good = []

    for i in row:
        try:
            word_array = fst_model.wv[i]
            dataset = pd.DataFrame(word_array.reshape((1, 300)))
            pred = clf_g.predict(dataset)
            if pred[0][0] == "B-GOOD":
                good.append(i)
        except Exception as ex:
            pass
    
    return ''.join(good)

In [ ]:
def create_submit_brand(row: pd.Series) -> pd.Series:
    """
    Формирует предсказания брендов
    :param row: токены
    :return: предсказанные бренды
    """    
    
    brand = []

    for i in row:
        try:
            word_array = fst_model.wv[i]
            dataset = pd.DataFrame(word_array.reshape((1, 300)))
            pred = clf_b.predict(dataset)
            if pred[0][0] == "B-BRAND":
                brand.append(i)
        except Exception as ex:
            pass
    
    return ''.join(brand)

In [ ]:
df_test["good"] = df_test["tokens"].apply(create_submit)
df_test["brand"] = df_test["tokens"].apply(create_submit_brand)

In [ ]:
df_fin = df_test.drop(['name', 'tokens'], axis=1, inplace=True)

In [ ]:
df_test.to_csv('itog.csv')